In [6]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

fname = os.path.join('CICIDS2017', 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_clean.csv')

flows = pd.read_csv(fname)
flows.tail()

c:\program files\python3x64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,21,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,flow_id,source_ip,source_port,destination_ip,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,external_ip
225740,192.168.10.17-192.168.10.50-37758-22-6,192.168.10.50,22,192.168.10.17,37758,6,2017-07-07T17:02:00,85,1,2,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,35294.1,42.5,7.778175,48.0,37.0,0.0,0.0,0.00000,0.0,0.0,48.0,48.0,0.0,48.0,48.0,0,0,0,0,32,64,11764.705880,23529.411760,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,2,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,2,0,243,290,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225741,192.168.10.14-23.10.108.151-59111-443-6,192.168.10.14,59111,23.10.108.151,443,6,2017-07-07T17:02:00,113,2,0,12,0,6,6,6.000000,0.00000,0,0,0.0,0.0,106195,17699.1,113.0,0.000000,113.0,113.0,113.0,113.0,0.00000,113.0,113.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,17699.115040,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,256,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225742,192.168.10.17-192.168.10.50-37759-22-6,192.168.10.17,37759,192.168.10.50,22,6,2017-07-07T17:02:00,115,1,1,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,17391.3,115.0,0.000000,115.0,115.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,32,32,8695.652174,8695.652174,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,1,0,290,243,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225743,162.213.33.50-192.168.10.51-443-59581-6,192.168.10.51,59581,162.213.33.50,443,6,2017-07-07T17:02:00,191310,3,2,148,0,148,0,49.333333,85.44784,0,0,0.0,0.0,773.614,26.1356,47827.5,54961.659690,95485.0,47.0,95825.0,47912.5,67175.85132,95413.0,412.0,95944.0,95944.0,0.0,95944.0,95944.0,0,0,0,0,104,72,15.681355,10.454237,0,148,24.666667,60.420747,3650.666667,0,0,0,1,0,0,0,0,0,29.6,49.333333,0.0,104,0,0,0,0,0,0,3,148,2,0,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225744,192.168.10.17-192.168.10.50-37759-22-6,192.168.10.50,22,192.168.10.17,37759,6,2017-07-07T17:02:00,81,1,2,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,37037,40.5,12.020815,49.0,32.0,0.0,0.0,0.00000,0.0,0.0,49.0,49.0,0.0,49.0,49.0,0,0,0,0,32,64,12345.679010,24691.358020,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,2,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,2,0,243,290,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN


In [7]:
# drop unused fields.
flows.drop(['flow_id'], axis=1, inplace=True)
flows.drop(['timestamp'], axis=1, inplace=True)
# TODO: Drop/replace all NaN values

In [8]:
import numpy as np
from keras.preprocessing.text import Tokenizer
# tokenize the LABELS
label_tokenizer = Tokenizer(num_words=20) 
label_tokenizer.fit_on_texts(flows['label'])

# Run the fitted tokenizer on the label column and save the encoded data as dataframe
enc_labels = label_tokenizer.texts_to_sequences(flows['label'])
enc_labels = np.concatenate(enc_labels).ravel()

# finally, drop the label column
flows.drop(['label'], axis=1, inplace=True)

In [9]:
# Generate OHE labels
df_source_ip = pd.get_dummies(flows['source_ip'], prefix='source_ip')
df_destination_ip = pd.get_dummies(flows['destination_ip'], prefix='destination_ip')
df_external_ip = pd.get_dummies(flows['external_ip'], prefix='external_ip')

#Append the label columns to the dataset
flows = pd.concat([flows, df_source_ip], axis=1)
flows = pd.concat([flows, df_destination_ip], axis=1)
flows = pd.concat([flows, df_external_ip], axis=1)

#drop the original columns
flows.drop(['source_ip'], axis=1, inplace=True)
flows.drop(['destination_ip'], axis=1, inplace=True)
flows.drop(['external_ip'], axis=1, inplace=True)


In [10]:
flows.head()

source_port  destination_port  protocol  flow_duration  total_fwd_packets  \
0        41936               443         6         143347                 47   
1        42970                80         6          50905                  1   
2        41944               443         6         143899                 46   
3        12886                53        17            313                  2   
4        41942               443         6         142605                 45   

   total_backward_packets  total_length_of_fwd_packets  \
0                      60                         1325   
1                       1                            0   
2                      58                         1325   
3                       2                           90   
4                      58                         1325   

   total_length_of_bwd_packets  fwd_packet_length_max  fwd_packet_length_min  \
0                       108751                    570                      0   
1                            0                      0                      0   
2                       110185                    570                      0   
3                          206                     45                     45   
4                       108751                    570                      0   

   fwd_packet_length_mean  fwd_packet_length_std  bwd_packet_length_max  \
0               28.191489             110.269756                   2896   
1                0.000000               0.000000                      0   
2               28.804348             111.407285                   4344   
3               45.000000               0.000000                    103   
4               29.444444             112.580588                   4344   

   bwd_packet_length_min  bwd_packet_length_mean  bwd_packet_length_std  \
0                      0             1812.516667             822.089741   
1                      0                0.000000               0.000000   
2                      0             1899.741379             942.998773   
3                    103              103.000000               0.000000   
4                      0             1875.017241             973.725897   

  flow_bytes_per_s flow_packets_per_s  flow_iat_mean  flow_iat_std  \
0      767898.8748        746.4404557    1352.330189   5067.810221   
1                0        39.28887143   50905.000000      0.000000   
2      774918.5192        722.7291364    1397.077670   5193.055313   
3       945686.901        12779.55272     104.333333    170.365294   
4      771894.3936         722.274815    1398.088235   5168.857742   

   flow_iat_max  flow_iat_min  fwd_iat_total  fwd_iat_mean  fwd_iat_std  \
0       23061.0           1.0       143347.0   3116.239130  7502.019684   
1       50905.0       50905.0            0.0      0.000000     0.000000   
2       23360.0           1.0       143899.0   3197.755556  7655.879725   
3         301.0           2.0           10.0     10.000000     0.000000   
4       23218.0           2.0       142557.0   3239.931818  7655.109026   

   fwd_iat_max  fwd_iat_min  bwd_iat_total  bwd_iat_mean  bwd_iat_std  \
0      23823.0          1.0       120283.0   2038.694915  6167.670541   
1          0.0          0.0            0.0      0.000000     0.000000   
2      23883.0          2.0       120525.0   2114.473684  6287.286944   
3         10.0         10.0            2.0      2.000000     0.000000   
4      23764.0         17.0       119443.0   2095.491228  6223.072915   

   bwd_iat_max  bwd_iat_min  fwd_psh_flags  bwd_psh_flags  fwd_urg_flags  \
0      24538.0          1.0              0              0              0   
1          0.0          0.0              0              0              0   
2      23836.0          2.0              0              0              0   
3          2.0          2.0              0              0              0   
4      23897.0          2.0              0              0              0   

   bwd_urg_flags  fwd_

In [11]:
# weed out all NaN and infinite values
flows.replace([np.inf, -np.inf], np.nan)
flows.fillna(inplace=True, value=0) # FIXME: 0 for now, find a better way

In [13]:
#convert all values to float-based numpy arrays.
# and while we're at it: normalize them
flows_nd = flows.astype('float64').values

In [14]:
# as the pandas infinity stff is seemingly not enough, check the numpy array once again
from numpy import inf
flows_nd[flows_nd == -inf] = 0
flows_nd[flows_nd == inf] = 0

In [15]:
print("Data has NaN:",np.any(np.isnan(flows_nd)))
print("Data has only finite values",np.all(np.isfinite(flows_nd)))

Data has NaN: False
Data has only finite values True


In [16]:
#normalize all dem stuff

from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
flows_scaled = min_max_scaler.fit_transform(flows_nd)

In [17]:
np.set_printoptions(threshold=np.nan)
print(flows_scaled[0])

[6.39912107e-01 6.76005616e-03 3.52941176e-01 1.19456728e-03
 2.38218540e-02 2.03942896e-02 7.23996241e-03 2.10254689e-02
 4.88013699e-02 0.00000000e+00 7.29027395e-03 1.64762595e-02
 2.47945205e-01 0.00000000e+00 3.12475936e-01 1.00320171e-01
 6.13251627e-03 4.00149288e-01 1.26479456e-05 7.32342517e-05
 1.92183332e-04 1.21495313e-07 1.19455833e-03 2.59686594e-05
 9.78099046e-05 1.98525000e-04 1.08333323e-07 1.00235833e-03
 1.69891243e-05 8.04129145e-05 2.04483333e-04 8.33333333e-09
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.04599452e-02 3.27601441e-02 1.09291905e-04 2.09282371e-04
 0.00000000e+00 2.47945205e-01 5.26231248e-01 2.28792537e-01
 5.23163562e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.42857143e-01 4.06941323e-01 7.29027395e-03
 3.12475936e-01 3.04599452e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.38218540e-02 7.239962

In [18]:
print(flows_scaled.shape)
print(type(flows_scaled))
print(type(flows_scaled[0]))

#find the maximum vocabulary size
voc_size = (flows_scaled.max()+1).astype('int64')
print(voc_size)

(225745, 4704)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
2


In [22]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

model = Sequential()
#model.add(Embedding(flows_nd.shape[-1], 32)) # embedding wants the [maximum integer index+1][1], I'm passing # of netflows.
model.add(Embedding(voc_size, 32)) 
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(flows_scaled, enc_labels,
                    epochs=1,
                    batch_size=128,
                    validation_split=0.2)

Train on 180596 samples, validate on 45149 samples
Epoch 1/1
 30080/180596 [===>..........................] - ETA: 1:24:37 - loss: -1.6633 - acc: 0.7964

KeyboardInterrupt: 